In [1]:
import pandas as pd
import redis
import csv
import time
from datetime import datetime
from fnmatch import fnmatch

In [2]:
r = redis.Redis()

In [3]:
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0)

In [6]:

with open('data/AirPollutionSeoul/Selected Data/instrument_status.csv', 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row['Instrument status code']
            item_info = {
                'Instrument status': row['Instrument status']
            }
            redis_client.hset(f'Instrument status: {key}', mapping=item_info)

In [226]:
with open('data/AirPollutionSeoul/Selected Data/merged_sensor.csv', 'r') as file:
        reader = csv.DictReader(file)
        inserting = {}
        
        with redis_client.pipeline() as pipe:
            for row in reader:
                timestamp = row['Timestamp']
                station_code= row['Station code']
                pollutant_code= row['Pollutant code']
                instrument_status_code = row['Instrument status code']
                pollutant_class_code = row['Pollutant class code']
                average_value = row['Average value']
                
                key = f"Measurement:{timestamp}:{station_code}:{pollutant_code}:{instrument_status_code}:{pollutant_class_code}"
                inserting[key] = average_value
                
                pipe.hset(f'{key}', mapping=item_info)
                pipe.sadd(f"measurements_by_timestamp:{timestamp}",key)
                pipe.sadd(f"measurements_by_station_code:{station_code}", key)
                pipe.sadd(f"measurements_by_pollutant_code:{pollutant_code}", key)
                pipe.sadd(f"measurements_by_instrument_status_code:{instrument_status_code}", key)
                pipe.sadd(f"measurements_by_pollutant_class_code:{pollutant_class_code}",key)
            pipe.execute()
        redis_client.mset(inserting)

In [40]:
#redis_client.flushall()

True

In [41]:
with open('data/AirPollutionSeoul/Selected Data/pollutant.csv', 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row['Pollutant code']
            item_info = {
                'pollutant': row['Pollutant name'],
                'unit of measurement code': row['Unit of measurement code']
            }
            redis_client.sadd('Pollutants',f'Pollutant code: {key}')
            redis_client.hset(f'Pollutant code: {key}', mapping=item_info)

In [42]:

with open('data/AirPollutionSeoul/Selected Data/pollutant_class.csv', 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row['Pollutant class code']
            item_info = {
                'Pollutant class':row['Pollutant class'],
            }
            redis_client.sadd('Pollutant_classes',f'Pollutant class: {key}')
            redis_client.hset(f'Pollutant class: {key}', mapping=item_info)

In [112]:
with open('data/AirPollutionSeoul/Selected Data/station.csv', 'r') as file:
        reader = csv.DictReader(file)#Station code,Station name(district),Address,Latitude,Longitude
        for row in reader:
            station_code = row['Station code']
            item_info = {
                'Station name': row['Station name/District'],
                'House number': row['House number'],
                'Street name': row['Street name'],
                'Latitude': row['Latitude'],
                'Longitude': row['Longitude'],
            }
            redis_client.sadd('Stations',f'Station: {station_code}')
            redis_client.hset(f'Station: {station_code}', mapping=item_info)

In [209]:

with open('data/AirPollutionSeoul/Selected Data/time.csv', 'r') as file:
        reader = csv.DictReader(file)#Station code,Station name(district),Address,Latitude,Longitude
        with redis_client.pipeline() as pipe:
            for row in reader:
                year= int(row['Year'])
                month= int(row['Month'])
                day= int(row['Day'])
                hour= int(row['Hour'])
                timestamp = row['Timestamp']
                item_info = {
                    'Year': year,
                    'Month': month,
                    'Day': day,
                    'Hour': hour,
                } 
                pipe.zadd('Timestamps_sorted',{timestamp: timestamp})
                pipe.hset(f'Timestamp: {timestamp}', mapping=item_info)
            pipe.execute()

In [45]:
with open('data/AirPollutionSeoul/Selected Data/unit_of_measurement.csv', 'r') as file:
        reader = csv.DictReader(file)#Station code,Station name(district),Address,Latitude,Longitude
        for row in reader:
            measurement_code = row['Unit of measurement code']
            item_info = {
                'Unit of measurement': row['Unit of measurement']
            }
            redis_client.sadd('Units_of_measurement',f'Unit of measurement code: {measurement_code}')
            redis_client.hset(f'Unit of measurement code: {measurement_code}', mapping=item_info)

PRASHALNICI

Сите записи што се направени од инструмент што е на поправка (instrument_status_code: 8)

In [360]:
status_8 = redis_client.smembers("measurements_by_instrument_status_code:8")
values = redis_client.mget(status_8)
for i,(k,v) in enumerate(zip(status_8,values)):
    print(f"key: {k.decode('utf-8')}, value: {v.decode('utf-8')}")
    if i == 499:
        break

key: Measurement:1523422800:113:6:8:0, value: 0.0
key: Measurement:1500368400:124:5:8:3, value: 1.9
key: Measurement:1569805200:110:6:8:0, value: 0.022
key: Measurement:1569603600:123:6:8:0, value: 0.005
key: Measurement:1519653600:103:6:8:0, value: -1.0
key: Measurement:1520046000:103:5:8:0, value: -1.0
key: Measurement:1519441200:122:1:8:0, value: 0.0
key: Measurement:1542668400:125:1:8:1, value: 0.021
key: Measurement:1570377600:110:5:8:3, value: 0.2
key: Measurement:1522519200:108:6:8:0, value: 0.03
key: Measurement:1569621600:121:9:8:0, value: 0.0
key: Measurement:1520035200:122:1:8:0, value: 0.0
key: Measurement:1531368000:101:1:8:0, value: 0.001
key: Measurement:1521493200:117:3:8:0, value: 0.0
key: Measurement:1515207600:105:5:8:0, value: 0.0
key: Measurement:1519970400:117:6:8:0, value: 0.0
key: Measurement:1569448800:121:8:8:0, value: 0.0
key: Measurement:1490126400:123:5:8:0, value: 0.0
key: Measurement:1521475200:117:1:8:0, value: 0.0
key: Measurement:1519596000:122:8:8:1, 

Сите записи каде просечното ниво (average_value) на загаденост со PM10 честички (pollutant_code: 8) е над дозволеното (50)

In [353]:
pollutant_8_keys = redis_client.smembers("measurements_by_pollutant_code:8")

average_values_8 = redis_client.mget(pollutant_8_keys)
for i,(k,v) in enumerate(zip(pollutant_8_keys,average_values_8)):
    val = v.decode('utf-8')
    if float(val)>50:
        val = v.decode('utf-8')
        print(f"key: {k}, value: {val}")

    # Print only the first 500 values
    if i == 499:
        break

key: b'Measurement:1523030400:110:8:0:3', value: 256.0
key: b'Measurement:1524168000:115:8:0:1', value: 52.0
key: b'Measurement:1517299200:124:8:9:1', value: 72.0
key: b'Measurement:1508616000:115:8:0:1', value: 65.0
key: b'Measurement:1550660400:103:8:0:1', value: 76.0
key: b'Measurement:1524186000:121:8:0:1', value: 66.0
key: b'Measurement:1563393600:101:8:0:1', value: 67.0
key: b'Measurement:1540209600:110:8:0:1', value: 53.0
key: b'Measurement:1492642800:119:8:0:2', value: 132.0
key: b'Measurement:1545238800:102:8:0:1', value: 58.0
key: b'Measurement:1517824800:107:8:9:1', value: 53.0
key: b'Measurement:1546603200:102:8:0:1', value: 69.0
key: b'Measurement:1547233200:124:8:0:2', value: 105.0
key: b'Measurement:1554026400:110:8:0:1', value: 74.0
key: b'Measurement:1556748000:125:8:0:2', value: 88.0
key: b'Measurement:1555376400:124:8:0:1', value: 57.0
key: b'Measurement:1514577600:121:8:0:2', value: 127.0
key: b'Measurement:1519398000:102:8:0:2', value: 107.0
key: b'Measurement:1556

Првите 100 записи каде што загадувањето е многу лошо (Pollutant class code: 3)

In [101]:
cursor, pollutant_class_3_keys = redis_client.sscan('measurements_by_pollutant_class_code:3', cursor=0,count=100)
average_values_3 = redis_client.mget(pollutant_class_3_keys)
for k,v in zip(pollutant_class_3_keys,average_values_3):
    val = v.decode('utf-8')
    print(f"key: {k.decode('utf-8')}, value: {val}")

key: Measurement:1562940000:118:5:0:3, value: 0.2
key: Measurement:1554667200:118:5:0:3, value: 0.5
key: Measurement:1514883600:117:5:0:3, value: 0.7
key: Measurement:1562670000:108:5:0:3, value: 0.5
key: Measurement:1495274400:112:5:0:3, value: 0.3
key: Measurement:1576180800:113:5:0:3, value: 0.5
key: Measurement:1483642800:117:5:0:3, value: 0.3
key: Measurement:1546218000:116:5:0:3, value: 0.8
key: Measurement:1507503600:124:5:0:3, value: 0.7
key: Measurement:1558177200:118:5:0:3, value: 0.3
key: Measurement:1492869600:102:5:0:3, value: 0.3
key: Measurement:1575568800:110:5:0:3, value: 0.4
key: Measurement:1488808800:116:5:0:3, value: 0.3
key: Measurement:1527026400:121:5:0:3, value: 0.3
key: Measurement:1543179600:116:5:0:3, value: 1.1
key: Measurement:1574823600:125:5:0:3, value: 0.5
key: Measurement:1545609600:123:5:0:3, value: 0.4
key: Measurement:1495526400:117:5:0:3, value: 0.4
key: Measurement:1496984400:106:5:0:3, value: 0.6
key: Measurement:1547161200:108:5:0:3, value: 1.1


Сите записи каде просечното ниво (average_value) на загаденост со PM10 честички (pollutant_code: 8) е над дозволеното (50).

Сите записи кои мерните станици им започнуваат со буквата 'S' и се направени во 2018 година

In [290]:
start_time = time.time()
station_keys = redis_client.smembers('Stations')

stations_with_s = []
for station in station_keys:
    if redis_client.hget(station, 'Station name').decode('utf-8')[0]=='S':
        name, code = station.decode('utf-8').split(" ")
        stations_with_s.append(code)


#za 2018 ni treba timestamp od pocetok i od kraj na 2018
start_timestamp = datetime(2018,1,1,0,0,0,0).timestamp()
end_timestamp = datetime(2018,12,31,23,59,59).timestamp()

timestamps_after = redis_client.zrangebyscore('Timestamps_sorted', start_timestamp, end_timestamp)
decoded_timestamps = set()
for timestamp in timestamps_after:
    decoded_timestamps.add(timestamp.decode('utf-8'))
    
keys = redis_client.keys('Measurement:*')
result_keys = []
for key in keys:
    key = key.decode('utf-8')
    _,timestamp,station,_,_,_ = key.split(":")
    if timestamp in decoded_timestamps and station in stations_with_s:
        result_keys.append(key)
values = redis_client.mget(result_keys)
for i, (k, v) in enumerate(zip(result_keys, values)):
    val = v.decode('utf-8')
    print(f"key: {k}, value: {val}")

    # Print only the first 500 values
    if i == 499:
        break
end_time = time.time()
total_time = end_time - start_time
print(f"Total execution time: {total_time} seconds")

key: Measurement:1538470800:105:3:0:0, value: 0.013
key: Measurement:1531533600:105:6:0:0, value: 0.001
key: Measurement:1525633200:111:6:0:0, value: 0.004
key: Measurement:1526396400:111:6:0:0, value: 0.053
key: Measurement:1524175200:122:8:0:1, value: 62.0
key: Measurement:1518764400:122:6:0:0, value: 0.008
key: Measurement:1522872000:107:9:0:0, value: 11.0
key: Measurement:1520038800:122:6:8:0, value: 0.0
key: Measurement:1526788800:122:1:0:0, value: 0.002
key: Measurement:1537912800:111:8:0:0, value: 11.0
key: Measurement:1516176000:107:3:0:1, value: 0.047
key: Measurement:1530165600:107:6:0:0, value: 0.012
key: Measurement:1531998000:124:5:0:3, value: 0.5
key: Measurement:1543395600:122:3:0:1, value: 0.034
key: Measurement:1534104000:111:6:0:0, value: 0.01
key: Measurement:1531281600:124:9:0:0, value: 9.0
key: Measurement:1538078400:122:6:0:0, value: 0.017
key: Measurement:1538064000:124:9:0:0, value: 2.0
key: Measurement:1522512000:111:3:0:1, value: 0.031
key: Measurement:1527624

Сите записи каде мерењето е направено во Mircrogram/m3

In [344]:
measurement_codes= redis_client.smembers('Units_of_measurement')

for measurement_code in measurement_codes:
    unit_of_measurement = redis_client.hget(measurement_code.decode('utf-8'),'Unit of measurement')
    if unit_of_measurement.decode('utf-8') ==  'Mircrogram/m3':
        _,microgram_code = measurement_code.decode('utf-8').split(": ")
        
pollutants_in_micrograms = []
pollutant_codes = redis_client.smembers('Pollutants')
for pollutant_code in pollutant_codes:
    pollutant = redis_client.hgetall(pollutant_code)
    if pollutant[b'unit of measurement code'].decode('utf-8') == microgram_code:
        _,tmp = pollutant_code.decode('utf-8').split(": ")
        pollutants_in_micrograms.append(tmp)

result_keys = []
result_values = []
for code in pollutants_in_micrograms:
    tkeys = redis_client.smembers(f'measurements_by_pollutant_code:{code}')
    result_keys += list(tkeys)
    result_values+= redis_client.mget(tkeys)

for i, (k, v) in enumerate(zip(result_keys, result_values)):
    val = v.decode('utf-8')
    print(f"key: {k}, value: {val}")
    if i == 499:
        break

key: b'Measurement:1568426400:121:8:4:0', value: -1.0
key: b'Measurement:1523030400:110:8:0:3', value: 256.0
key: b'Measurement:1566838800:115:8:0:1', value: 50.0
key: b'Measurement:1562864400:120:8:0:0', value: 10.0
key: b'Measurement:1496556000:116:8:0:1', value: 32.0
key: b'Measurement:1530698400:125:8:0:0', value: 30.0
key: b'Measurement:1556874000:113:8:1:0', value: 0.0
key: b'Measurement:1487638800:113:8:0:0', value: 26.0
key: b'Measurement:1509757200:109:8:0:1', value: 45.0
key: b'Measurement:1531501200:104:8:0:0', value: 27.0
key: b'Measurement:1524168000:115:8:0:1', value: 52.0
key: b'Measurement:1494882000:104:8:4:0', value: -1.0
key: b'Measurement:1546218000:108:8:0:1', value: 36.0
key: b'Measurement:1573398000:106:8:0:1', value: 32.0
key: b'Measurement:1568322000:121:8:4:0', value: -1.0
key: b'Measurement:1521230400:107:8:0:0', value: 21.0
key: b'Measurement:1512770400:103:8:0:1', value: 32.0
key: b'Measurement:1487793600:115:8:0:1', value: 44.0
key: b'Measurement:151729920

Првите 100 записи (час, ден и тип на загадувач) од 2019 година каде што доминантен загадувач е PM 10 (Pollutant code: 8)

In [414]:
pollutants = redis_client.smembers('Pollutants')
for pollutant_code in pollutants:
    pollutant = redis_client.hgetall(pollutant_code)
    if pollutant[b'pollutant'].decode('utf-8') == 'PM10':
        _,PM10_code = pollutant_code.decode('utf-8').split(": ")

start_timestamp = datetime(2019,1,1,0,0,0,0).timestamp()
end_timestamp = datetime(2019,12,31,23,59,59).timestamp()

timestamps_after = redis_client.zrangebyscore('Timestamps_sorted', start_timestamp, end_timestamp)
decoded_timestamps = set()
for timestamp in timestamps_after:
    decoded_timestamps.add(timestamp.decode('utf-8'))
result_keys = []
hours = []
days = []
cursor = 0

for timestamp in timestamps_after:
    timestamp = timestamp.decode('utf-8')
    cursor, tkeys = redis_client.sscan(cursor = cursor,name = f"measurements_by_timestamp:{timestamp}",count=100)
    for tkey in tkeys:
        decoded_key = tkey.decode('utf-8')
        _,_,_,check_t,_,_ = decoded_key.split(":")
        #print(check_t)
        if check_t == PM10_code:
            result_keys.append(decoded_key)
            day = redis_client.hget(f'Timestamp: {timestamp}','Day')
            hour = redis_client.hget(f'Timestamp: {timestamp}','Hour')
            days.append(day.decode('utf-8'))
            hours.append(hour.decode('utf-8'))
        if len(result_keys) == 100:
            break
    if len(result_keys) == 100:
            break

result_values = redis_client.mget(result_keys)

for k, v,h,d in zip(result_keys, result_values,hours,days):
    val = v.decode('utf-8')
    print(f"key: {k}, value: {val}, hour: {h}, day: {d}, pollutant: PM10")

key: Measurement:1546300800:116:8:0:1, value: 55.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:118:8:0:1, value: 34.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:117:8:0:1, value: 43.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:108:8:0:1, value: 38.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:114:8:0:1, value: 48.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:112:8:0:1, value: 43.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:119:8:0:1, value: 47.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:104:8:0:1, value: 34.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:102:8:0:1, value: 38.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:110:8:0:1, value: 32.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:120:8:0:1, value: 36.0, hour: 1, day: 1, pollutant: PM10
key: Measurement:1546300800:115:8:0:1, value: 56.0, hour: 1, day:

Просечна измерена загаденост по мерна станица

In [51]:
station_totals = {}
stations = redis_client.smembers('Stations')
station_codes = []

for station in stations:
    _,code = station.decode('utf-8').split(": ")
    station_codes.append(code)
    
with redis_client.pipeline() as pipe:
    for station_code in station_codes:
        measurement_keys = redis_client.smembers(f'measurements_by_station_code:{station_code}')
        measurements = redis_client.mget(measurement_keys)
        total_value = 0
        count = 0
        
        for measurement in measurements:
                total_value += float(measurement.decode('utf-8'))
                count += 1
        if count > 0:
            average = total_value / count
            station_totals[station_code] = average

for station_code, average in station_totals.items():
    print(f"Station code: {station_code}, Average pollution: {average}")


Station code: 108, Average pollution: 12.622908752074181
Station code: 112, Average pollution: 10.177376257767516
Station code: 115, Average pollution: 11.497696435809925
Station code: 116, Average pollution: 13.079798859856192
Station code: 119, Average pollution: 13.766460115351679
Station code: 114, Average pollution: 10.87365038317179
Station code: 110, Average pollution: 10.068451601487842
Station code: 111, Average pollution: 12.063555375390543
Station code: 121, Average pollution: 12.184435198053384
Station code: 104, Average pollution: 11.291621545074586
Station code: 120, Average pollution: 11.090599584154834
Station code: 106, Average pollution: 12.852490882298977
Station code: 124, Average pollution: 11.749481530096743
Station code: 122, Average pollution: 14.026334594495681
Station code: 109, Average pollution: 10.240557250040501
Station code: 103, Average pollution: 9.944693100653485
Station code: 101, Average pollution: 10.241933171613725
Station code: 102, Average pollut

Колку вкупно пати било измерено секој од 4-те типа на загаденост во период од 01.04.2017 до 15.10.2017

In [11]:
start_timestamp = datetime(2017,4,1,0,0,0,0).timestamp()
end_timestamp = datetime(2017,10,11,23,59,59).timestamp()

timestamps_after = redis_client.zrangebyscore('Timestamps_sorted', start_timestamp, end_timestamp)

decoded_timestamps = [timestamp.decode('utf-8') for timestamp in timestamps_after]
pollutant_class_counts = {}

for timestamp in decoded_timestamps:
    measurements = redis_client.smembers(f'measurements_by_timestamp:{timestamp}')
    for measurement in measurements:
        _,_,_,_,_,pollutant_class = measurement.decode('utf-8').split(":")
        pollutant_class_counts[pollutant_class] = pollutant_class_counts.get(pollutant_class,0)+1



for class_code, count in pollutant_class_counts.items():
    pollutant_class = redis_client.hget(f'Pollutant class: {class_code}', "Pollutant class").decode('utf-8')
    print(f"Pollutant Class: {pollutant_class}, Count: {count}")



Pollutant Class: Very bad(Red), Count: 115943
Pollutant Class: Good(Blue), Count: 414396
Pollutant Class: Normal(Green), Count: 140260
Pollutant Class: Bad(Yellow), Count: 24201


Сите записи (ден, сума колку пати тој ден е направено мерење каде се мери во PM 10) од 2019 година

In [50]:
start_timestamp = datetime(2019, 1, 1, 0, 0, 0, 0).timestamp()
end_timestamp = datetime(2019, 12, 31, 23, 59, 59, 59).timestamp()

# Fetch timestamps within the specified range
timestamps_after = redis_client.zrangebyscore('Timestamps_sorted', start_timestamp, end_timestamp)

# Decode timestamps to strings
decoded_timestamps = [timestamp.decode('utf-8') for timestamp in timestamps_after]

# Initialize a dictionary to store the sum of PM10 pollution values for each day
pm10_sum_by_month_and_date = {}

for timestamp in decoded_timestamps:
    measurements = redis_client.smembers(f'measurements_by_timestamp:{timestamp}')
    pm10_measurements = [s for s in measurements if fnmatch(s.decode('utf-8'),"Measurement:*:*:8:*:*")]
    date = datetime.fromtimestamp(int(timestamp))
    key = f"Month: {date.month} Date: {date.day}"
    
    for measurement in pm10_measurements:
        if key in pm10_sum_by_month_and_date:
            pm10_sum_by_month_and_date[key] += 1
        else:
            pm10_sum_by_month_and_date[key] = 1

for key, pm10_sum in pm10_sum_by_month_and_date.items():
    print(f"{key}, PM10 Sum: {pm10_sum}")
    

Month: 1 Date: 1, PM10 Sum: 575
Month: 1 Date: 2, PM10 Sum: 600
Month: 1 Date: 3, PM10 Sum: 600
Month: 1 Date: 4, PM10 Sum: 600
Month: 1 Date: 5, PM10 Sum: 600
Month: 1 Date: 6, PM10 Sum: 600
Month: 1 Date: 7, PM10 Sum: 600
Month: 1 Date: 8, PM10 Sum: 600
Month: 1 Date: 9, PM10 Sum: 600
Month: 1 Date: 10, PM10 Sum: 600
Month: 1 Date: 11, PM10 Sum: 600
Month: 1 Date: 12, PM10 Sum: 600
Month: 1 Date: 13, PM10 Sum: 600
Month: 1 Date: 14, PM10 Sum: 600
Month: 1 Date: 15, PM10 Sum: 600
Month: 1 Date: 16, PM10 Sum: 600
Month: 1 Date: 17, PM10 Sum: 600
Month: 1 Date: 18, PM10 Sum: 600
Month: 1 Date: 19, PM10 Sum: 600
Month: 1 Date: 20, PM10 Sum: 600
Month: 1 Date: 21, PM10 Sum: 600
Month: 1 Date: 22, PM10 Sum: 600
Month: 1 Date: 23, PM10 Sum: 600
Month: 1 Date: 24, PM10 Sum: 600
Month: 1 Date: 25, PM10 Sum: 600
Month: 1 Date: 26, PM10 Sum: 600
Month: 1 Date: 27, PM10 Sum: 600
Month: 1 Date: 28, PM10 Sum: 600
Month: 1 Date: 29, PM10 Sum: 600
Month: 1 Date: 30, PM10 Sum: 600
Month: 1 Date: 31, 